In [4]:
import pandas as pd

In [5]:
pd.read_csv("log/describe_all.csv")

,global_size,nx,nvx,kernel,error_mean,error_std,runtime_mean,runtime_std,cellspersec_mean,cellspersec_std,throughput_mean,throughput_std,gpu
0,65536,128,512,BasicRange2D,1.710180e-06,2.232128e-22,0.006017,0.004660,0.019014,0.013119,0.304228,0.209897,0
1,262144,256,1024,BasicRange2D,1.053090e-07,1.395080e-23,0.007582,0.003865,0.043484,0.020491,0.695746,0.327851,0
2,524288,512,1024,BasicRange2D,3.278930e-09,0.000000e+00,0.008713,0.002343,0.063939,0.015985,1.023018,0.255757,0
3,65536,128,512,BasicRange2D,1.710180e-06,2.232128e-22,0.007043,0.004515,0.015413,0.011930,0.246601,0.190886,0
4,262144,256,1024,BasicRange2D,1.053090e-07,1.395080e-23,0.004855,0.001474,0.057495,0.013644,0.919925,0.218307,0
5,524288,512,1024,BasicRange2D,3.278930e-09,0.000000e+00,0.007579,0.001147,0.070812,0.012062,1.132996,0.192990,0
6,65536,128,512,BasicRange1D,1.710180e-06,2.232128e-22,0.009032,0.008496,0.017144,0.014285,0.274309,0.228566,0
7,262144,256,1024,BasicRange1D,1.053090e-07,1.395080e-23,0.005499,0.002206,0.052826,0.015476,0.845209,0.247621,0
8,524288,512,1024,BasicRange1D,3.278930e-09,0.000000e+00,0.008058,0.003592,0.074653,0.025186,1.194448,0.402982,0
9,65536,128,512,BasicRange1D,1.710180e-06,2.232128e-22,0.013803,0.014301,0.009537,0.006740,0.152590,0.107847,0
